In [1]:
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/"

record = pd.read_csv(data+'watched.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': 'category'
})

train_full = pd.read_csv(data+'train_withcv.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': np.int8,
    'cv': np.int8
})

test = pd.read_csv(data+'test.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16
})

In [2]:
u1 = pd.crosstab(record.user_id, record.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)

In [3]:
ratelut = {
    'dislike':1.0,
    'like':3.0,
    'love':4.0,
    'neutral':2.0
}
record['score']=record.rating.map(lambda x: ratelut[x]).astype(float)
u2 = record[['user_id', 'score']].groupby(by='user_id')['score'].agg(['mean', 'std']).rename(columns={'mean':'user_mean', 'std':'user_std'})

In [5]:
i1 = pd.crosstab(record.work_id, record.rating).add_prefix('work_').apply(lambda r: r/r.sum(), axis=1)
i2 = record[['work_id', 'score']].groupby(by='work_id')['score'].agg(['mean', 'std']).add_prefix('item_')

In [16]:
valid = train_full[train_full.cv==1]
train = train_full[train_full.cv.isin([2,3])]
u3 = pd.crosstab(train.user_id, train.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)
i3 = pd.crosstab(train.work_id, train.rating).add_prefix('item_').apply(lambda r: r/r.sum(), axis=1)

In [17]:
train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(u3, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left').\
merge(i3, left_on='work_id', right_index=True, how='left')

In [18]:
train = train.drop(['user_id', 'work_id', 'cv'], axis=1)
train.to_csv(data+"train_3.csv", index=False)

In [19]:
valid = valid.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(u3, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left').\
merge(i3, left_on='work_id', right_index=True, how='left')
valid = valid.drop(['user_id', 'work_id', 'cv'], axis=1)

In [20]:
valid.to_csv(data+"valid_3.csv", index=False)

In [21]:
train = train_full
u3 = pd.crosstab(train.user_id, train.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)
i3 = pd.crosstab(train.work_id, train.rating).add_prefix('item_').apply(lambda r: r/r.sum(), axis=1)
train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(u3, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left').\
merge(i3, left_on='work_id', right_index=True, how='left')
train = train.drop(['user_id', 'work_id', 'cv'], axis=1)
train.to_csv(data+"train_0.csv", index=False)

In [22]:
test = test.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(u3, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left').\
merge(i3, left_on='work_id', right_index=True, how='left')
test = test.drop(['user_id', 'work_id'], axis=1)
test.to_csv(data+"test_0.csv", index=False)

In [23]:
test

,user_dislike,user_like,user_love,user_neutral,user_mean,user_std,user_0,user_1,work_dislike,work_like,work_love,work_neutral,item_mean,item_std,item_0,item_1
0,0.119565,0.728261,0.000000,0.152174,2.608696,0.694666,0.400000,0.600000,0.279412,0.225490,0.009804,0.485294,1.965686,0.738657,0.560000,0.440000
1,0.048276,0.841379,0.000000,0.110345,2.793103,0.512301,0.217391,0.782609,0.121212,0.686869,0.010101,0.181818,2.585859,0.714430,1.000000,0.000000
2,0.027778,0.833333,0.000000,0.138889,2.805556,0.467177,0.800000,0.200000,0.279412,0.225490,0.009804,0.485294,1.965686,0.738657,0.560000,0.440000
3,0.152597,0.629870,0.000000,0.217532,2.477273,0.745979,0.676056,0.323944,0.062500,0.666667,0.027778,0.243056,2.659722,0.638652,0.470588,0.529412
4,0.031373,0.450980,0.000000,0.517647,2.419608,0.554516,0.569231,0.430769,0.134146,0.707317,0.024390,0.134146,2.621951,0.747675,0.666667,0.333333
5,0.033333,0.700000,0.000000,0.266667,2.666667,0.546672,0.000000,1.000000,0.040523,0.733333,0.104575,0.121569,2.901961,0.615727,0.275862,0.724138
6,0.048701,0.681818,0.001623,0.267857,2.636364,0.576710,0.855856,0.144144,0.092105,0.565789,0.171053,0.171053,2.815789,0.828018,0.666667,0.333333
7,0.112299,0.668449,0.000000,0.219251,2.556150,0.688463,0.500000,0.500000,0.096774,0.677419,0.032258,0.193548,2.645161,0.709384,0.500000,0.500000
8,0.051282,0.871795,0.000000,0.076923,2.820513,0.506370,0.000000,1.000000,0.123173,0.586639,0.096033,0.194154,2.655532,0.815851,0.500000,0.500000
9,0.031169,0.724675,0.200000,0.044156,3.093506,0.600854,0.581081,0.418919,0.000000,0.666667,0.000000,0.333333,2.666667,0.516398,0.500000,0.500000
